In [ ]:
#
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 22 14:48:16 2019

@author: Administrator
"""

import wx
import wx.grid
import os
import shutil
from data_info import GetData
from GetAbstract import GetAbstract
from MyDataFrame import MyDataFrame
from ShowAbstract import ShowAbstract
from WordsCloud import WordsCloud
from ShowCloud import ShowCloud
from OpenPDF import OpenPDF
from ShowPath import ShowPath

# 自定义窗口类MyFrame
class MyFrame(wx.Frame):
    column_names =['标题','类别','关键字' ]
    data=[['','','']]
    def __init__(self):
        super().__init__(parent=None, title='个人电子文件检索系统', size=(1000,700))
        self.Centre()  # 设置窗口居中
        self.panel = wx.Panel(self)
        self.SetBackgroundColour('#4DB3B3')
        #最底层box
        vbox0 = wx.BoxSizer(wx.VERTICAL)
        #上下两个box
        vbox1 = wx.BoxSizer()#水平
        vbox2 = wx.BoxSizer()#水平
        #1.在最底层添加标题       
        header_text = wx.StaticText(self.panel,label='个人电子文件检索系统',style=wx.TE_CENTER)
        font = wx.Font(18,wx.DEFAULT,wx.NORMAL,wx.NORMAL)
        header_text.SetFont(font)

        #2.在上层box添加内容
        openfiles_button = wx.Button(self.panel,id=1,label='打开')#打开文件夹
        openfiles_button.Bind(wx.EVT_BUTTON,self.OnOpen)       
        self.openfiles_text = wx.TextCtrl(self.panel,value='',style=wx.TE_LEFT)
        
        #搜索的文本框
        self.search_text = wx.TextCtrl(self.panel,value='',style=wx.TE_LEFT)
        search_button = wx.Button(self.panel,id=1,label='搜索')
        search_button.Bind(wx.EVT_BUTTON,self.SearchFiles)
     
        #3.向下层box添加内容   
        #下面box中的box
        vbox3 = wx.BoxSizer(wx.VERTICAL)#垂直
        vbox4 = wx.BoxSizer(wx.VERTICAL)       
        #网格
        
        self.grid = wx.grid.Grid(self.panel)
        self.grid.CreateGrid(len(self.data), len(self.data[0]))#行、列
        self.GridSetData(self.data,self.column_names)

        self.grid.SetColSize(0,330)
        self.grid.SetColSize(1,80)
        self.grid.SetColSize(2,240)
        
        # 设置行和列自定调整
        #self.grid.AutoSize()
        #绑定事件，当用户在行或列的标签区域敲击鼠标左键时触发该事件
        self.Bind(wx.grid.EVT_GRID_LABEL_LEFT_CLICK, self.OnLabelLeftClick)  
        self.Bind(wx.grid.EVT_GRID_LABEL_LEFT_DCLICK,self.OnLableLeftDClick)
        
        self.path_show = wx.StaticText(self.panel,label='当前文件夹路径：')
               
        #下面右侧控件
        #分类下拉列表
        hbox5 = wx.BoxSizer()
        fenlei_text = wx.StaticText(self.panel,label='分类：')
        list1 = ['A_All','Computer','Economy','Enviornment','Agriculture','Art','Politics','Sports']
        #ch1下拉列表，value为默认值，choices对应一个列表，style=wx.CB_SORT使选择列表中的元素按字母顺序显示。
        fenlei = wx.Choice(self.panel, -1, choices=list1, style=wx.CB_SORT)
        self.Bind(wx.EVT_CHOICE, self.on_choice, fenlei)#绑定事件
        
        #按钮      
        showpath_button = wx.Button(self.panel,label='查看路径')
        self.Bind(wx.EVT_BUTTON, self.showpath,showpath_button)
        
        delete_button = wx.Button(self.panel,label='删除')
        self.Bind(wx.EVT_BUTTON, self.deletefile,delete_button)
        move_button = wx.Button(self.panel,label='移动')
        self.Bind(wx.EVT_BUTTON, self.movefile,move_button)
        ciyun_button = wx.Button(self.panel,label='词云')
        self.Bind(wx.EVT_BUTTON, self.show_cloud,ciyun_button)
        abstract_button = wx.Button(self.panel,label='摘要')
        self.Bind(wx.EVT_BUTTON, self.get_abstract,abstract_button)
        #similar_button= wx.Button(self.panel,label='相似文档')
             
        #向上层box添加内容       
        #将打开文件夹添加到上层box
        vbox1.Add(openfiles_button,1,flag=wx.EXPAND)       
        vbox1.Add(self.openfiles_text,3,flag= wx.LEFT | wx.EXPAND,border=10)
        #将搜索框添加到上层box
        vbox1.Add(self.search_text,5,flag= wx.LEFT | wx.EXPAND,border=300)
        vbox1.Add(search_button,1,flag=wx.EXPAND | wx.LEFT, border=10)
        
        
        #想下层box添加内容
        vbox3.Add(self.path_show,1,flag=wx.EXPAND | wx.LEFT, border=10)
        vbox3.Add(self.grid,10,flag= wx.EXPAND)
        hbox5.Add(fenlei_text,1,flag=wx.EXPAND)
        hbox5.Add(fenlei,3,flag=wx.EXPAND)
        vbox4.Add(hbox5,flag= wx.EXPAND)
        vbox4.Add(showpath_button,flag= wx.EXPAND | wx.TOP,border = 150)
        vbox4.Add(delete_button,flag= wx.EXPAND | wx.TOP,border = 30)
        vbox4.Add(move_button,flag= wx.EXPAND | wx.TOP,border = 30)
        
        vbox4.Add(ciyun_button,flag= wx.EXPAND | wx.TOP,border = 30)
        vbox4.Add(abstract_button,flag= wx.EXPAND | wx.TOP, border = 30)
        #vbox4.Add(similar_button,flag= wx.EXPAND | wx.TOP, border = 20)
    
    
        #向下层box添加内容
        
        vbox2.Add(vbox3,5,flag= wx.EXPAND)
        vbox2.Add(vbox4,1,flag= wx.EXPAND | wx.ALL, border = 20)
        
     
        #向底层box添加内容       
        vbox0.Add(header_text,flag = wx.TOP | wx.EXPAND | wx.CENTER,border = 20) #将标题添加到底层box
        vbox0.Add(vbox1,1,flag=wx.ALL | wx.EXPAND,border=20)
        vbox0.Add(vbox2,10,flag=wx.ALL | wx.EXPAND,border=20)
        self.panel.SetSizer(vbox0)
        
    
    def GridSetData(self,data,column_names):
        #添加数据
        for row in range(len(data)):
            for col in range(len(data[row])):
                self.grid.SetColLabelValue(col, column_names[col])#设置列标题
                self.grid.SetCellValue(row, col, data[row][col])#设置单元格内容
                
                #SetRowLabelValue()，SetColLabelValue()和SetCellValue()，它们实际上设置显示在网格中的值
                #SetCellValue()方法要求一个行索引、一个列索引和一个值。而其它两个方法要求一个索引和一个值。
              
    def OnOpen(self,event):
        dlg = wx.DirDialog(None,message='打开文件夹',style=wx.DD_DEFAULT_STYLE)
        
        if dlg.ShowModal() == wx.ID_OK:
            self.file = dlg.GetPath()          
            #print(self.file)#file为文件夹路径名
            self.openfiles_text.SetLabelText(self.file)# 在下方写入路径
            self.path_show.SetLabelText('当前文件夹路径：'+self.file)
            dlg.Destroy()

        #向后台传入路径file，获取data
            gd = GetData()
            self.data =[]
            self.data= gd.getInitData()
        #print(self.data)
        #print(len(self.data))
        
            self.grid.AppendRows(numRows = len(self.data) - 1)
            self.GridSetData(self.data,self.column_names)
        # 设置行和列自定调整
        #self.grid.AutoSize()
  
            
    def SearchFiles(self,event):
        str_search = self.search_text.GetValue()
        print(str_search)
        #print('搜索文件')
        gd = GetData()
        simfiles = gd.getSearchFiles(str_search)
        print(len(self.data)) #1.50
        print(len(simfiles))# computer 5
        print(simfiles)       
        if len(simfiles) == 0:
            nulldata = [['','','']]
            if len(self.data) == 1:
                self.data = nulldata
            else:               
                numrows = len(self.data) - 1 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
                self.data = nulldata
        else:
            if len(self.data) > len(simfiles):    
                numrows = len(self.data) - len(simfiles) 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
            elif len(self.data) < len(simfiles):
                numrows = len(simfiles) - len(self.data)
                self.grid.AppendRows(numRows = numrows)
            else:
                pass
            self.data = simfiles
        self.GridSetData(self.data,self.column_names)
        
      
    #网格点击事件
    def OnLabelLeftClick(self, event):
        print("RowIdx：{0}".format(event.GetRow()))#行索引
        self.rowidx = event.GetRow()
        print("ColIdx：{0}".format(event.GetCol()))#列索引
        #print(self.data[event.GetRow()])
        self.fileselect = self.data[event.GetRow()]
        print(self.fileselect)
        event.Skip()#确保还可以响应其他事件处理
 
    
    def OnLableLeftDClick(self,event):
        #print("RowIdx：{0}".format(event.GetRow()))#行索引
        #print("ColIdx：{0}".format(event.GetCol()))#列索引
        #print(self.data[event.GetRow()])
        self.fileselect = self.data[event.GetRow()]
        
        print(self.file) #文件夹路径
        #self.fileselect[0] 文件名
        
        allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
        for i in range(len(allfiles)):
            if allfiles[i].find(self.fileselect[0]) != -1:
                filepath = allfiles[i]
                print(filepath)
                op = OpenPDF()
                op.experimentGuide(1,filepath)
                #os.popen(r'"C:\Program Files (x86)\Foxit Software\Foxit Reader\FoxitReader.exe" filepath ')
    

    def get_abstract(self,event):
        try:
            filename = self.fileselect[0]
            print(filename)
            mdf = MyDataFrame()
            df = mdf.new_DataFrame()
            for i in range(len(df)):
                if df.filename[i] == filename:
                    file_content = df.content[i]

            #获取摘要
            ga = GetAbstract()
            abstract_contents = ga.readAbstract(filename)
            if len(abstract_contents) == 0:
                abstract_contents = ga.generateAbstract(file_content)
            else:
                pass
            print(abstract_contents)
            
            dialog = ShowAbstract(abstract_contents)
            dialog.Show()
            
        except:
            pass
    
    def showpath(self,event):       
        try:
            filename = self.fileselect[0]
            
            allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
            for i in range(len(allfiles)):
                if allfiles[i].find(filename) != -1:
                    filepath = allfiles[i]
                    print(filepath)
            
            dialog = ShowPath(filepath)
            dialog.Show()
            
        except:
            pass
     
    def deletefile(self,event):
        try:
            dlg = wx.MessageDialog(None,message = '是否确定删除？', caption='删除文件',style=wx.YES_NO|wx.ICON_QUESTION)
            if dlg.ShowModal() == wx.ID_YES:
                filename = self.fileselect[0]            
                allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
                for i in range(len(allfiles)):
                    if allfiles[i].find(filename) != -1:
                        filepath = allfiles[i]
                        print(filepath)               
                os.remove(filepath) #删除pdf文件
                os.remove('C:/EdmsData/txt/'+filename+'.txt')#删除txt文件
                
                gd = GetData()
                sydata = gd.deleteOneFile(filename)
                
                self.grid.DeleteRows(pos = self.rowidx,numRows = 1)
                print('------------------------------------------1ok')
                self.data = sydata    
                self.GridSetData(self.data,self.column_names)
                print('yes')
            else:
                print('no')
            dlg.Destroy()           
        except:
            pass
     
    def movefile(self,event):
        try:
            dlg = wx.DirDialog(None,message='',style=wx.DD_DEFAULT_STYLE)       
            if dlg.ShowModal() == wx.ID_OK: 
                newfile = dlg.GetPath()
                filename = self.fileselect[0]            
                allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
                for i in range(len(allfiles)):
                    if allfiles[i].find(filename) != -1:
                        filepath = allfiles[i]
                        print(filepath)
                oldpath = filepath
                newpath = newfile + '/' + filename+'.pdf'
                if(newfile.find(self.file)):# 新路径在原路径下，直接移动
                    shutil.move(oldpath,newpath)
                    print('move finish') 
                else: #移动后删除该文件数据
                    shutil.move(oldpath,newpath)
                    os.remove('C:/EdmsData/txt/'+filename+'.txt')#删除txt文件
                    gd = GetData()
                    sydata = gd.deleteOneFile(filename)

                    self.grid.DeleteRows(pos = self.rowidx,numRows = 1)
                    print('------------------------------------------1ok')
                    self.data = sydata    
                    self.GridSetData(self.data,self.column_names)
                    print('move finish')
                
            dlg.Destroy()

       
            
        except:
            pass
            
    
    

        #遍历文件夹及其子文件夹
    def list_all_files(self,rootdir):
        _files = []
        list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.list_all_files(path))
            if os.path.isfile(path):
                _files.append(rootdir +'\\'+list[i])
        return _files
            
            
        
    def show_cloud(self,event):
        try:
            filename = self.fileselect[0]
            print(filename)
            wc = WordsCloud()
            wc.saveCloud(filename)
            sc = ShowCloud(filename)
            sc.Show()
        except:
            pass


    #分类按钮下拉选择
    def on_choice(self, event):
        print('选择 {0}'.format(event.GetString()))
        catagory_select = event.GetString()
        gd = GetData()
        catagory_data = gd.getCatagoryData(catagory_select)
        print(len(self.data)) #1.50
        print(len(catagory_data))# computer 5
        #print(self.data)
        print(catagory_data)
        if len(catagory_data) == 0:
            nulldata = [['','','']]
            if len(self.data) == 1:
                self.data = nulldata
            else:               
                numrows = len(self.data) - 1 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
                self.data = nulldata
        else:
            if len(self.data) > len(catagory_data):    
                numrows = len(self.data) - len(catagory_data) 
                self.grid.DeleteRows(pos = 1,numRows = numrows)
            elif len(self.data) < len(catagory_data):
                numrows = len(catagory_data) - len(self.data)
                self.grid.AppendRows(numRows = numrows)
            else:
                pass
            self.data = catagory_data
        self.GridSetData(self.data,self.column_names)
        

class App(wx.App):

    def OnInit(self):
        # 创建窗口对象
        frame = MyFrame()
        frame.Show()
        return True


if __name__ == '__main__':
    app = App()
    app.MainLoop()  # 进入主事件循环



Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.606 seconds.
Prefix dict has been built succesfully.


数据挖掘
数据挖掘技术在电子商务管理中的应用_高宇:7.038040540558164
数据挖掘技术在电网运营监控平台建设中的应用_李呓瑾:6.969612770706014
遗传算法的数据挖掘技术在医疗大数据中的应用研究_陈萌:6.934375488922107
数据挖掘技术在网络营销中的应用分析_王冬屏:6.709699213058658
云计算视域下数据挖掘技术_杨继武:6.6749632174374485
软件工程数据挖掘的发展趋势_乔磊:6.5643128688292025
文本挖掘与中文文本挖掘模型研究:6.028929264212778
电子商务用户数据的知识融合研究_王越:5.906336127516999
卫星数据挖掘节点级并行与优化方法_鲍军鹏:5.744957219473484
基于大数据的商业智能在电商数据分析中的应用_钱丹丹:4.702358991214121
60
10
[['数据挖掘技术在电子商务管理中的应用_高宇', 'Computer', '数据挖掘,数据,电子商务,技术,用户'], ['数据挖掘技术在电网运营监控平台建设中的应用_李呓瑾', 'Computer', '数据挖掘,电网,监控,数据,运营'], ['遗传算法的数据挖掘技术在医疗大数据中的应用研究_陈萌', 'Computer', '数据挖掘,数据,医疗,算法,技术'], ['数据挖掘技术在网络营销中的应用分析_王冬屏', 'Computer', '数据挖掘,电子商务,客户,技术,网络'], ['云计算视域下数据挖掘技术_杨继武', 'Computer', '数据挖掘,云计算,技术,数据,借助'], ['软件工程数据挖掘的发展趋势_乔磊', 'Computer', '数据挖掘,软件工程,技术,信息,数据'], ['文本挖掘与中文文本挖掘模型研究', 'Computer', '文本挖掘,文本,中文,语义,挖掘'], ['电子商务用户数据的知识融合研究_王越', 'Computer', '用户,电子商务,知识,数据,数据挖掘'], ['卫星数据挖掘节点级并行与优化方法_鲍军鹏', 'Computer', '聚类,GPU,线程,优化,数据挖掘'], ['基于大数据的商业智能在电商数据分析中的应用_钱丹丹', 'Computer', '数据,中药饮片,客户,企业,质心']]
选择

RowIdx：0
ColIdx：-1
['云计算视域下数据挖掘技术_杨继武', 'Computer', '数据挖掘,云计算,技术,数据,借助']
C:\TMdata\PDF\数据挖掘\云计算视域下数据挖掘技术_杨继武.pdf
云计算视域下数据挖掘技术_杨继武
RowIdx：3
ColIdx：-1
['国外对计算机软件知识产权的保护_陈红', 'Computer', '软件,保护,版权法,计算机软件,计算机程序']
国外对计算机软件知识产权的保护_陈红
国外对计算机软件知识产权的保护_陈红
摘要 :本文评述了美国 、日本 、欧洲和发展中国家对计算机软件的知识产权法律保护的现状 , 揭示了发展中国家包括中国在这方面与发达国家之间悬殊的差距 , 为我国计算机软件的立法和司法提供借鉴和帮助 。
选择 Sports
20
5
[['2017年广州市体育产业增加值425_76亿元_黄心豪', 'Sports', '体育产业,2017,广州市,体育,2016'], ['体育总局举行党组理论学习中心组_扩大_学习会议_顾宁', 'Sports', '习近平,青年,学习,总书记,运动员'], ['改革为中国羽毛球积蓄能量_周圆', 'Sports', '2018,混双,冠军,女单,羽协'], ['激发体育社会组织活力_陈伟', 'Sports', '体育,组织,社会,长三角,发展'], ['用榜样的力量玩跨界_刘昕彤', 'Sports', '漫威,跨界,体育明星,NASA,粉丝']]
RowIdx：0
ColIdx：-1
['2017年广州市体育产业增加值425_76亿元_黄心豪', 'Sports', '体育产业,2017,广州市,体育,2016']
2017年广州市体育产业增加值425_76亿元_黄心豪
11
[(6, 0.11068832123954232), (1, 0.10880926337854559), (8, 0.10459318027877455), (0, 0.10120254711402765), (7, 0.10049999495809638), (10, 0.083593362343509142), (5, 0.082877059698055505), (2, 0.080003559118508261), (4, 0.07897185632257428),

In [2]:
    def deletefile(self,event):
        try:
            dlg = wx.MessageDialog(None,message = '是否确定删除？', caption='删除文件',style=wx.YES_NO|wx.ICON_QUESTION)
            if dlg.ShowModal() == wx.ID_YES:
                filename = self.fileselect[0]            
                allfiles =self.list_all_files(self.file) #所有文件，包含所在路径
                for i in range(len(allfiles)):
                    if allfiles[i].find(filename) != -1:
                        filepath = allfiles[i]
                        print(filepath)               
                os.remove(filepath) #删除pdf文件
                os.remove('C:/EdmsData/txt/'+filename+'.txt')#删除txt文件
                self.grid.DeleteRows(pos =self.rowidx,numRows = 1)
                gd = GetData()
                self.data = []
                self.data = gd.getInitData()
                
                print('yes')
            else:
                print('no')
            dlg.Destroy()
                
            
        except:
            pass

In [1]:
import pandas as pd
df = pd.DataFrame()
print(df)

Empty DataFrame
Columns: []
Index: []
